In [1]:
import tensorflow as tf

In [3]:
import tensorflow.contrib.eager as tfe

When you enable eager execution, operations execute immediately and 
return their values to Python without requiring a Session.run(). 
For example, to multiply two matrices together, we write this:

In [4]:
tfe.enable_eager_execution()

x = [[2.]]

m = tf.matmul(x, x)

In [5]:
print(m)

tf.Tensor([[ 4.]], shape=(1, 1), dtype=float32)


Dynamic models can be built with Python flow control. 
Here's an example of the Collatz conjecture using TensorFlow’s arithmetic operations:

In [6]:
a = tf.constant(12)
counter = 0
while not tf.equal(a, 1):
    if tf.equal(a % 2, 0):
        a = a / 2
    else:
        a = 3 * a + 1
    print(a)
    

tf.Tensor(6.0, shape=(), dtype=float64)
tf.Tensor(3.0, shape=(), dtype=float64)
tf.Tensor(10.0, shape=(), dtype=float64)
tf.Tensor(5.0, shape=(), dtype=float64)
tf.Tensor(16.0, shape=(), dtype=float64)
tf.Tensor(8.0, shape=(), dtype=float64)
tf.Tensor(4.0, shape=(), dtype=float64)
tf.Tensor(2.0, shape=(), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64)


Most TensorFlow users are interested in automatic differentiation. 
Because different operations can occur during each call, we record all forward operations to a tape, 
which is then played backwards when computing gradients. 
After we've computed the gradients, we discard the tape. 
If you’re familiar with the autograd package, the API is very similar. For example:

In [7]:
def square(x):
    return tf.multiply(x, x)

grad = tfe.gradients_function(square)

print(square(3.))    # [9.]
print(grad(3.))      # [6.]

tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=112, shape=(), dtype=float32, numpy=6.0>]


The gradients_function call takes a Python function square() as an argument and 
returns a Python callable that computes the partial derivatives of square() with respect to its inputs. 
So, to get the derivative of square() at 3.0, invoke grad(3.0), which is 6. 
The same gradients_function call can be used to get the second derivative of square:

In [8]:
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])

print(gradgrad(3.))  # [2.]

[<tf.Tensor: id=140, shape=(), dtype=float32, numpy=2.0>]


As we noted, control flow can cause different operations to run, such as in this example.

In [9]:
def abs(x):
    return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(2.0))  # [1.]
print(grad(-2.0)) # [-1.]

[<tf.Tensor: id=150, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=161, shape=(), dtype=float32, numpy=-1.0>]


Users may want to define custom gradients for an operation, or for a function. 
This may be useful for multiple reasons, including providing a more efficient or 
more numerically stable gradient for a sequence of operations. Here is an example that illustrates 
the use of custom gradients. Let's start by looking at the function log(1 + ex), 
which commonly occurs in the computation of cross entropy and log likelihoods.

In [10]:
def log1pexp(x):
    return tf.log(1 + tf.exp(x))
grad_log1pexp = tfe.gradients_function(log1pexp)

# The gradient computation works fine at x = 0.
print(grad_log1pexp(0.))
# [0.5]
# However it returns a `nan` at x = 100 due to numerical instability.
print(grad_log1pexp(100.))
# [nan]

[<tf.Tensor: id=173, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=185, shape=(), dtype=float32, numpy=nan>]


We can use a custom gradient for the above function that analytically simplifies the gradient expression. 
Notice how the gradient function implementation below reuses an expression (tf.exp(x)) 
that was computed during the forward pass, 
making the gradient computation more efficient by avoiding redundant computation.

In [12]:
@tfe.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad
grad_log1pexp = tfe.gradients_function(log1pexp)

# Gradient at x = 0 works as before.
print(grad_log1pexp(0.))
# [0.5]
# And now gradient computation at x=100 works as well.
print(grad_log1pexp(100.))
# [1.0]

[<tf.Tensor: id=198, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=211, shape=(), dtype=float32, numpy=1.0>]


Models can be organized in classes. 
Here's a model class that creates a (simple) two layer network that can classify 
the standard MNIST handwritten digits.

In [14]:
class MNISTModel(tfe.Network):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = self.track_layer(tf.layers.Dense(units=10))
        self.layer2 = self.track_layer(tf.layers.Dense(units=10))
    def call(self, input):
        """Actually runs the model."""
        result = self.layer1(input)
        result = self.layer2(result)
        return result
    

We recommend using the classes (not the functions) in tf.layers since they create and contain model parameters (variables). Variable lifetimes are tied to the lifetime of the layer objects, so be sure to keep track of them. Why are we using tfe.Network? A Network is a container for layers and is a tf.layer.Layer itself, allowing Network objects to be embedded in other Network objects. It also contains utilities to assist with inspection, saving, and restoring. Even without training the model, we can imperatively call it and inspect the output:

In [15]:
# Let's make up a blank input image
model = MNISTModel()
batch = tf.zeros([1, 1, 784])
print(batch.shape)
# (1, 1, 784)
result = model(batch)
print(result)
# tf.Tensor([[[ 0.  0., ...., 0.]]], shape=(1, 1, 10), dtype=float32)

(1, 1, 784)
tf.Tensor([[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]], shape=(1, 1, 10), dtype=float32)


Note that we do not need any placeholders or sessions. The first time we pass in the input, the sizes of the layers’ parameters are set. To train any model, we define a loss function to optimize, calculate gradients, and use an optimizer to update the variables. First, here's a loss function:

In [16]:
def loss_function(model, x, y):
    y_ = model(x)
    return tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)

And then, our training loop:

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
for (x, y) in tfe.Iterator(dataset):
    grads = tfe.implicit_gradients(loss_function)(model, x, y)
    optimizer.apply_gradients(grads)

NameError: name 'dataset' is not defined

implicit_gradients() calculates the derivatives of loss_function with respect to all the TensorFlow variables used during its computation. We can move computation to a GPU the same way we’ve always done with TensorFlow:



In [18]:
with tf.device("/gpu:0"):
    for (x, y) in tfe.Iterator(dataset):
        optimizer.minimize(lambda: loss_function(model, x, y))

NameError: name 'dataset' is not defined

(Note: We're shortcutting storing our loss and directly calling the optimizer.minimize, but you could also use the apply_gradients() method above; they are equivalent.)